In [128]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests

print('Libraries imported.')

Libraries imported.


# Creating the DataFrame of Canadian Postalcodes

### I install the beautifulsoup library to scrape the table from the wikipedia page.

In [10]:
!pip install beautifulsoup4

     |████████████████████████████████| 122kB 5.5MB/s eta 0:00:01


### Note
>I import the beautifulSoup class from bs4 and with the request library get the underlying html of the page. I then create a beautifulsoup object and print a prettified version of it (I will only a part of this so scrolling doesn't become a problem). I scanned through this html that i called soup and found the class pointing to the table.

In [129]:
from bs4 import BeautifulSoup

url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(url)

print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8689


### Note:
>I then use the find method to extract the portion of the html relating to the table and the findAll and text function to take out the text from the html into variables header(a list) and body(a list of lists)

In [130]:
table= soup.find('table',{'class':'wikitable sortable'})

headers = [j.text.rstrip() for j in table.findAll('tr')[0].findAll('th')]
body= [[j.text.rstrip() for j in i.findAll('td')] for i in table.findAll('tr')[1:]]

### I use the filter method to extract all rows that have Boroughs. I also check and verify that no row with a borough has a Not assigned Neighborhood (Hence there is no need to process anything as regards that).

In [144]:
new_body = filter(lambda x: x[1]!= 'Not assigned',body)
print(f'Number of cells with a borough but a Not assigned Neighborhoods:{len(list(filter(lambda x: x[1] != "Not assigned" and x[2]== "Not assigned",body)))}')

Number of cells with a borough but a Not assigned Neighborhoods:0


### I loaded my data with its columns into a dataframe object.

In [145]:
neigh_df = pd.DataFrame(new_body,columns=headers)
neigh_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Finally, I output the shape of the dataframe.

In [141]:
neigh_df.shape

(103, 3)

# Adding the Geographical Data

### I used the pd.read_csv method to read the csv data from the link provided and check its shape.

In [146]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.shape

(103, 3)

### I used the Pandas merge method to merge the neighbourhood data and the geographical data into one dataframe called df.

In [152]:
df = pd.merge(neigh_df,geo_df,on = 'Postal Code')
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
